## _*Using Qiskit Aqua for graph partition problems*_

Here we consider a simplified graph partition problem.
Consider a graph $G = (V, E)$, where $V$ denotes the set of n vertices and $E$ the set of edges. 
The objective of graph partition is to partition $G$ into two sets of the same size (let us assume we have even number of vertices), 
while minimizing the capacity of the edges across the two sets.

We will go through three examples to show (1) how to run the optimization in the non-programming way, (2) how to run the optimization in the programming way, (3) how to run the optimization with the VQE.
We will omit the details for the support of CPLEX, which are explained in other notebooks such as maxcut.

Note the objective_value below is defined as the the number of crossing edges. The goal is to minimize this value.


### The problem and the brute-force method

the graph involved in our example is as follows. The graph is in the adjacent matrix form.

In [1]:
import numpy as np
from qiskit import Aer
from qiskit_aqua import run_algorithm
from qiskit_aqua.input import EnergyInput
from qiskit_aqua.translators.ising import graphpartition
from qiskit_aqua.algorithms import ExactEigensolver

np.random.seed(100)
num_nodes = 4
w = graphpartition.random_graph(num_nodes, edge_prob=0.8, weight_range=10)
print(w)

[[ 0.  4.  5.  3.]
 [ 4.  0. -5.  7.]
 [ 5. -5.  0.  0.]
 [ 3.  7.  0.  0.]]


the brute-force method is as follows. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a vertex is either 0 (meaning the vertex is in the first partition) or 1 (meaning the vertex is in the second partition). We print the binary assignment that satisfies the definition of the graph partition and corresponds to the minimial number of crossing edges.

In [2]:
def brute_force():
    # use the brute-force way to generate the oracle
    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

    L = num_nodes
    max = 2**L
    minimal_v = np.inf
    for i in range(max):
        cur = bitfield(i, L)

        how_many_nonzero = np.count_nonzero(cur)
        if how_many_nonzero * 2 != L:  # not balanced
            continue

        cur_v = graphpartition.objective_value(np.array(cur), w)
        if cur_v < minimal_v:
            minimal_v = cur_v
    return minimal_v

sol = brute_force()
print("objective value computed by the brute-force method is", sol)

objective value computed by the brute-force method is 3


### Part I: run the optimization in the non-programming way

In [3]:
qubit_op, offset = graphpartition.get_graphpartition_qubitops(w)
algo_input = EnergyInput(qubit_op)

params = {
    'problem': {'name': 'ising'},
    'algorithm': {'name': 'ExactEigensolver'}
}
result = run_algorithm(params, algo_input)
x = graphpartition.sample_most_likely(result['eigvecs'][0])
# check against the oracle
ising_sol = graphpartition.get_graph_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 0, 1])
print("objective value computed by ExactEigensolver is", graphpartition.objective_value(x, w))

objective value computed by ExactEigensolver is 3.0


### Part II: run the optimization in the programming way

In [4]:
algo = ExactEigensolver(algo_input.qubit_op, k=1, aux_operators=[])
result = algo.run()
x = graphpartition.sample_most_likely(result['eigvecs'][0])
# check against the oracle
ising_sol = graphpartition.get_graph_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 0, 1])
print("objective value computed by the ExactEigensolver is", graphpartition.objective_value(x, w))

objective value computed by the ExactEigensolver is 3.0


### Part III: run the optimization with VQE

In [5]:
algorithm_cfg = {
    'name': 'VQE',
    'operator_mode': 'matrix'
}

optimizer_cfg = {
    'name': 'SPSA',
    'max_trials': 300
}

var_form_cfg = {
    'name': 'RY',
    'depth': 5,
    'entanglement': 'linear'
}

params = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg,
    'variational_form': var_form_cfg
}
backend = Aer.get_backend('statevector_simulator')
result = run_algorithm(params, algo_input, backend=backend)
x = graphpartition.sample_most_likely(result['eigvecs'][0])
# check against the oracle
ising_sol = graphpartition.get_graph_solution(x)
np.testing.assert_array_equal(ising_sol, [1, 0, 0, 1])
print("objective value computed by VQE is", graphpartition.objective_value(x, w))

objective value computed by VQE is 3.0
